# 5. 사람이 말하는 단어를 인공지능 모델로 구분해보자

## 1. 들어가며
인공지능 개인비서 시리! 다들 들어보았을 것이다.

시리는 인공지능 음성인식 서비스의 대표적인 사례이다. '시리야 날씨정보 알려줘~'라고 말하면 신기하게 날씨정보를 알려준다. 우리 생활 속에서 쉽게 찾아볼 수 있는 음성인식 서비스의 예이다.

<img src="./image/siri.png" />

이번 시간에는 시리처럼 사람의 목소리를 듣고 그것을 텍스트로 변환하는 모델을 만들어보자. 음성은 지금까지 다뤄보지 않은 도메인이라 낯설고 어려울 수 있다. 하지만 차근차근 잘 따라오면 절대 어렵지 않으니, 차분히 따라가자.

**음성(speech)**은 사람의 언어가 들어있는 소리를 말하는 것이고, **오디오(audio)**는 사람 목소리 이외의 모든 소리를 통칭해서 일컫는 것이다. 생활소리, 동물울음소리, 클래식, 악기소리 등등이 포함된다.<br>
음성의 경우 일정한 주파수 영역에만 해당하기 때문에 일반적인 소리보다는 처리하기가 수월해진다. 일반적인 오디오를 다루는 것은 좀 더 까다롭다. 하지만 기본적인 원리는 비슷하다.

오늘 음성 분류기 모델이 음성을 처리하는 과정을 살펴볼 것이다. 그러려면 먼저 오디오 데이터가 가지는 기본적인 원리와 특성에 대해 알아보아야한다. 아울러 디지털 데이터밖에 처리하지 못하는 컴퓨터가 오디오 데이터를 처리하려면 어떤 방식으로 아날로그 오디오 데이터를 디지털화해서 처리하는지도 함께 알아볼 것이다.

### **학습 목표**

---

* Audio 형태의 데이터를 다루는 방법에 대해서 알아보기
* Wav 파일의 형태와 원리를 이해하기
* 오디오데이터를 다른 다양한 형태로 변형시켜보기
* 차원이 다른 데이터에 사용가능한 classification 모델 직접 제작해보기

### **목차**

---

1. 들어가며
2. 음성데이터란?
3. Train / Test 데이터셋 구성하기
4. Wav classification 모델 구현
5. Skip-Connection 모델을 추가해보자
6. Spectrogram
7. 프로젝트: Spectrogram classification 모델 구현

## 3. Train/Test 데이터셋 구성하기

### **Label data 처리**
---

현재 단어의 정답은 Text 형태로 이뤄져있다. 학습을 위해서는 Text 데이터를 학습가능한 형태로 만들어줘야 한다.

아래는 구분해야할 label 목록이다.

**`['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go' ]`**

이외 데이터들은 'unknown', 'silence'로 분류되어 있다.

In [ ]:
target_list = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']

label_value = target_list
label_value.append('unknown')
label_value.append('silence')

print('LABEL : ', label_value)

new_label_value = dict()
for i, l in enumerate(label_value):
    new_label_value[l] = i
label_value = new_label_value

print('Indexed LABEL : ', new_label_value)

Text로 이루어진 라벨 데이터를 학습에 사용하기 위해서 index 형태로 바꿔주는 작업을 하였다.

int로 이뤄진 index 작업을 통해서 Label data를 더 쉽게 사용할 수 있다.

In [ ]:
temp = []
for v in speech_data["label_vals"]:
    temp.append(label_value[v[0]])
label_data = np.array(temp)

label_data

### 학습을 위한 데이터 분리
---

sklearn의 train_test_split 함수를 이용해 train data와 test data를 분리하겠다.<br>
test_size의 인자를 조절해주면, 설정해 준 값만큼 Test dataset의 비율을 조정할 수 있다.



In [ ]:
from sklearn.model_selection import train_test_split

sr = 8000
train_wav, test_wav, train_label, test_label = train_test_split(speech_data["wav_vals"], 
                                                                label_data, 
                                                                test_size=0.1,
                                                                shuffle=True)
print(train_wav)

train_wav = train_wav.reshape([-1, sr, 1]) # add channel for CNN
test_wav = test_wav.reshape([-1, sr, 1])
print("✅")

나눠진 데이터셋을 확인해 보겠다.

In [ ]:
print("train data : ", train_wav.shape)
print("train labels : ", train_label.shape)
print("test data : ", test_wav.shape)
print("test labels : ", test_label.shape)
print("✅")

### Hyper-parameters setting
---

학습을 위한 하이퍼파라미터를 설정해준다. 모델 체크포인트 저장을 위한 체크포인트의 경로를 설정해준다.<br>
후에 모델 체크포인트 Callback 함수를 설정하거나, 모델을 불러올때 사용한다.

In [ ]:
batch_size = 32
max_epochs = 10

# the save point
checkpoint_dir = os.getenv('HOME')+'/aiffel/speech_recognition/models/wav'

checkpoint_dir

### **Data setting**
---

**`tf.data.Dataset`**을 이용해서 데이터셋을 구성하겠다. Tensorflow에 포함된 이 데이터셋 관리 패키지는 데이터셋 전처리, 배치처리 등을 쉽게 할 수 있도록 해 준다.

**`tf.data.Dataset.from_tensor_slices`** 함수에 return 받길 원하는 데이터를 튜플 (data, label) 형태로 넣어서 사용할 수 있다.

**`map`** 함수는 dataset이 데이터를 불러올때마다 동작시킬 데이터 전처리 함수를 매핑해 주는 역할을 한다. 첫번째 **`map`** 함수는 **`from_tensor_slice`** 에 입력한 튜플 형태로 데이터를 받으며 return 값으로 어떤 데이터를 반환할지 결정한다.**`map`** 함수는 중첩해서 사용이 가능하다.

아래와 같이, **`map`** 함수에 넘겨줄 데이터 전처리 함수를 작성해보자.

In [ ]:
def one_hot_label(wav, label):
    label = tf.one_hot(label, depth=12)
    return wav, label
print("✅")

**`tf.data.Dataset`** 함수를 구성하겠다.<br>
batch는 dataset에서 제공하는 튜플 형태의 데이터를 얼마나 가져올지 결정하는 함수이다.

In [ ]:
import tensorflow as tf

# for train
train_dataset = tf.data.Dataset.from_tensor_slices((train_wav, train_label))
train_dataset = train_dataset.map(one_hot_label)
train_dataset = train_dataset.repeat().batch(batch_size=batch_size)
print(train_dataset)

# for test
test_dataset = tf.data.Dataset.from_tensor_slices((test_wav, test_label))
test_dataset = test_dataset.map(one_hot_label)
test_dataset = test_dataset.batch(batch_size=batch_size)
print(test_dataset)
print("✅")

## 4. Wave classification 모델 구현

### Model
---
Audio 데이터는 1차원 데이터이기 때문에 데이터 형식에 맞도록 모델을 구성해주어야 한다.<br>
**`Conv1D`** layer를 이용해서 모델을 구성해보겠다.<br>
Conv, batch norm, dropout, dense layer 등을 이용해 모델을 구성해보도록 하자.

In [ ]:
from tensorflow.keras import layers

input_tensor = layers.Input(shape=(sr, 1))

x = layers.Conv1D(32, 9, padding='same', activation='relu')(input_tensor)
x = layers.Conv1D(32, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)

x = layers.Conv1D(64, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(64, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)

x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)

x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.MaxPool1D()(x)
x = layers.Dropout(0.3)(x)

x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

output_tensor = layers.Dense(12)(x)

model_wav = tf.keras.Model(input_tensor, output_tensor)

model_wav.summary()

### Loss
---
현재 라벨이 될 수 있는 12개의 단어 class를 가지고 있다.<br>
해당 class를 구분하기 위해서는 multi-class classification이 필요하며, 이를 수행하기 위한 Loss로 Categorical Cross-Entropy loss를 사용하겠다.

In [ ]:
optimizer=tf.keras.optimizers.Adam(1e-4)
model_wav.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             optimizer=optimizer,
             metrics=['accuracy'])
print("✅")

### Training
---

#### Callback

* **`model.fit`** 함수를 이용할 때, callback 함수를 이용해서 학습 중간 중간 원하는 동작을 하도록 설정할 수 있다.
* 모델을 재사용하기위해서 모델 가중치를 저장하는 callback 함수를 추가해보겠다.

**`Model Checkpoint callback`**은 모델을 학습을 진행하며, **`fit`** 함수내 다양한 인자를 지정해 모니터하며 동작하게 설정할 수 있다.<br>
현재 모델은 validation loss를 모니터하며, loss가 낮아지면 모델 파라미터를 저장하도록 구성되어 있다.

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='val_loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)
print("✅")

아래는 모델 학습 코드이다. 이전 스텝의 하이퍼파라미터 세팅에서 **`batch_size=32`**, **`max_epochs=10`**으로 세팅한 경우라면 30분 가량 소요될 것이다. 메모리 사용량에 주의하며 적절히 하이퍼파라미터 세팅을 조절하자. 메모리가 부족하다면 **`batch_size`**를 작게 조절해 주는게 좋다.

In [ ]:
#30분 내외 소요 (메모리 사용량에 주의해 주세요.)
history_wav = model_wav.fit(train_dataset, epochs=max_epochs,
                    steps_per_epoch=len(train_wav) // batch_size,
                    validation_data=test_dataset,
                    validation_steps=len(test_wav) // batch_size,
                    callbacks=[cp_callback]
                    )
print("✅")

### 학습 결과 Plot
---
model.fit 함수는 학습 동안의 결과를 return해준다.<br>
return 값을 기반으로 loss와 accuracy를 그래프로 표현하겠다.<br>
fit 함수에서 전달 받은 Loss와 Accuracy의 값을 이용해 모델이 어떻게 학습되고 있는지 볼 수 있다.<br>
train loss와 val_loss의 차이가 커지는 경우 오버피팅이 일어나는 것이기 때문에 이를 수정할 필요가 있다.

출력된 그래프를 기반으로 모델의 학습이 어떻게 진행됐는지 확인해볼 수 있다.

In [ ]:
import matplotlib.pyplot as plt

acc = history_wav.history['accuracy']
val_acc = history_wav.history['val_accuracy']

loss=history_wav.history['loss']
val_loss=history_wav.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
print("✅")

### Evaluation
---
Test dataset을 이용해서 모델의 성능을 평가한다.

실습삼아 checkpoint callback 함수가 저장한 weight를 다시 불러와서 테스트 준비를 해보겠다.

In [ ]:
model_wav.load_weights(checkpoint_dir)
print("✅")

Test data을 이용하여 모델의 예측값과 실제값이 얼마나 일치하는지 확인하겠다.

In [ ]:
results = model_wav.evaluate(test_dataset)
print("✅")

In [ ]:
# loss
print("loss value: {:.3f}".format(results[0]))
# accuracy
print("accuracy value: {:.4f}%".format(results[1]*100))
print("✅")

### Model Test
---
Test data 셋을 골라 직접 들어보고 모델의 예측이 맞는지 확인해보자.

In [ ]:
inv_label_value = {v: k for k, v in label_value.items()}
batch_index = np.random.choice(len(test_wav), size=1, replace=False)

batch_xs = test_wav[batch_index]
batch_ys = test_label[batch_index]
y_pred_ = model_wav(batch_xs, training=False)

print("label : ", str(inv_label_value[batch_ys[0]]))

ipd.Audio(batch_xs.reshape(8000,), rate=8000)

위에서 확인해본 테스트셋의 라벨과 우리 모델의 실제 prediction 결과를 비교해보자.

In [ ]:
if np.argmax(y_pred_) == batch_ys[0]:
    print("y_pred: " + str(inv_label_value[np.argmax(y_pred_)]) + '(Correct!)')
else:
    print("y_pred: " + str(inv_label_value[np.argmax(y_pred_)]) + '(Incorrect!)')
print("✅")

## 5. Skip-Connection model을 추가해보자

### Skip-Connection model 구현
---
이전 스텝에서 우리는 Conv1D 기반의 간단한 분류 모델을 구현해서 학습 및 테스트를 진행해 보았다. 간단한 모델임에도 정확도가 나쁘지 않았을 것이다.

하지만 여러분들은 이미지처리 모델을 다루면서 ResNet 등 skip-connection을 활용한 모델들이 훨씬 안정적으로 높은 성능을 낼 수 있음을 배워왔을 것이다. 그렇다면 이번 음성처리 모델에 적용해도 비슷한 개선 효과를 낼 수 있지 않을까?

기존의 모델을 skip-connection이 추가된 모델로 변경해 학습을 진행해보겠다.

<img src="./image/skipconnection.png" />
<center>[ResNet 논문에 나오는 Skip-connection 개념]</center>

그림에서 보듯이 위쪽의 데이터가 레이어를 뛰어넘어 레이어를 통과한 값에 더해주는 형식으로 구현됨을 확인할 수 있다.<br>
Concat을 이용한 방식으로 구현하면 된다.

**`tf.concat([#layer output tensor, layer output tensor#], axis=#)`**

우리가 사용하는 데이터가 1차원 audio 데이터이기 때문에 1차원 데이터를 처리하는 모델을 구성해보겠다.

In [ ]:
input_tensor = layers.Input(shape=(sr, 1))

x = layers.Conv1D(32, 9, padding='same', activation='relu')(input_tensor)
x = layers.Conv1D(32, 9, padding='same', activation='relu')(x)
skip_1 = layers.MaxPool1D()(x)

x = layers.Conv1D(64, 9, padding='same', activation='relu')(skip_1)
x = layers.Conv1D(64, 9, padding='same', activation='relu')(x)
x = tf.concat([x, skip_1], -1)
skip_2 = layers.MaxPool1D()(x)

x = layers.Conv1D(128, 9, padding='same', activation='relu')(skip_2)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(128, 9, padding='same', activation='relu')(x)
x = tf.concat([x, skip_2], -1)
skip_3 = layers.MaxPool1D()(x)

x = layers.Conv1D(256, 9, padding='same', activation='relu')(skip_3)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = layers.Conv1D(256, 9, padding='same', activation='relu')(x)
x = tf.concat([x, skip_3], -1)
x = layers.MaxPool1D()(x)
x = layers.Dropout(0.3)(x)

x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

output_tensor = layers.Dense(12)(x)

model_wav_skip = tf.keras.Model(input_tensor, output_tensor)

model_wav_skip.summary()

모델 구성만 달라졌을 뿐, 그 외 Task구성이나 데이터셋 구성, 훈련 과정은 동일하다.

In [ ]:
optimizer=tf.keras.optimizers.Adam(1e-4)
model_wav_skip.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             optimizer=optimizer,
             metrics=['accuracy'])
print("✅")

In [ ]:
# the save point
checkpoint_dir = os.getenv('HOME')+'/aiffel/speech_recognition/models/wav_skip'

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='val_loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)
print("✅")

In [ ]:
#30분 내외 소요
history_wav_skip = model_wav_skip.fit(train_dataset, epochs=max_epochs,
                    steps_per_epoch=len(train_wav) // batch_size,
                    validation_data=test_dataset,
                    validation_steps=len(test_wav) // batch_size,
                    callbacks=[cp_callback]
                    )
print("✅")

학습결과의 시각화 및 evaluation 과정도 동일하다.

In [ ]:
import matplotlib.pyplot as plt

acc = history_wav_skip.history['accuracy']
val_acc = history_wav_skip.history['val_accuracy']

loss=history_wav_skip.history['loss']
val_loss=history_wav_skip.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
print("✅")

In [ ]:
# Evaluation 

model_wav_skip.load_weights(checkpoint_dir)
results = model_wav_skip.evaluate(test_dataset)

# loss
print("loss value: {:.3f}".format(results[0]))
# accuracy
print("accuracy value: {:.4f}%".format(results[1]*100))
print("✅")

In [ ]:
# Test 

inv_label_value = {v: k for k, v in label_value.items()}
batch_index = np.random.choice(len(test_wav), size=1, replace=False)

batch_xs = test_wav[batch_index]
batch_ys = test_label[batch_index]
y_pred_ = model_wav_skip(batch_xs, training=False)

print("label : ", str(inv_label_value[batch_ys[0]]))

ipd.Audio(batch_xs.reshape(8000,), rate=8000)

위에서 확인해본 테스트셋의 라벨과 우리 모델의 실제 prediction 결과를 비교해보자.

In [ ]:
if np.argmax(y_pred_) == batch_ys[0]:
    print("y_pred: " + str(inv_label_value[np.argmax(y_pred_)]) + '(Correct!)')
else:
    print("y_pred: " + str(inv_label_value[np.argmax(y_pred_)]) + '(Incorrect!)')
print("✅")

## 7. 프로젝트: Spectrogram classification 모델 구현

그래서 오늘은 방금 든 궁금증을 해결해 보는 것으로 프로젝트를 진행해보자.

오늘 실습에서 1차원 Waveform 데이터를 입력받아 Text 라벨을 출력하는 모델을 기본 버전과 Skip-connection 버전으로 나누어 학습시켜 보았다. 이번에는 2차원 Spectrogram 데이터를 입력받아 위 모델과 동일한 역할을 수행하는 모델을 아래 제시된 단계와 같이 수행해보자. 이번에도 마찬가지로 기본 버전과 Skip-connection 버전으로 나누어 각각 진행해보자.<br>
모델 구조를 제외하고는 실습에서 제시된 것과 거의 동일하게 진행될 것이다.

### 1. 데이터 처리와 분류
---

* 라벨 데이터 처리하기
* **`sklearn`**의 **`train_test_split`**함수를 이용하여 train, test 분리

### 2. 학습을 위한 하이퍼파라미터 설정
---

### 3. 데이터셋 구성
---
* **`tf.data.Dataset`**을 이용
* from_tensor_slices 함수에 return 받길 원하는 데이터를 튜플 (data, label) 형태로 넣어서 사용
* map과 batch를 이용한 데이터 전처리


* 주의 : waveform을 spectrogram으로 변환하기 위해 추가로 사용하는 메모리 때문에 이후 메모리 부족 현상을 겪게 될수도 있다.
* **`tf.data.Dataset`**이 생성된 이후, 아래 예시와 같이 wav 데이터나 spectrogram 데이터를 담아둔 메모리 버퍼를 비워 주면 도움이 된다.

```python
del speech_data
del spec_data
```

### 4. 2차원 Spectrogram 데이터를 처리하는 모델 구성
---
* 2차원 Spectrogram 데이터의 시간축 방향으로 Conv1D layer를 적용, 혹은 Conv2D layer를 적용 가능
* batchnorm, dropout, dense layer 등을 이용
* 12개의 단어 class를 구분하는 loss를 사용하고 Adam optimizer를 사용
* 모델 가중치를 저장하는 checkpoint callback 함수 추가
* 다양한 모델의 실험을 진행해 보시기 바랍니다.

### 5. 학습 후, 학습이 어떻게 진행됐는지 그래프로 출력
---
* loss, accuracy를 그래프로 표현

### 6. Test dataset을 이용해서 모델의 성능을 평가
---
* 저장한 weight 불러오기
* 모델의 예측값과 정답값이 얼마나 일치하는지 확인